In [1]:
# import packages
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

device = ("cpu")

In [2]:
# define a Dataset class
class MyDataset(Dataset):
    def __init__(self, features, target):
        self.y = target
        self.X = features
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx, :], self.y[idx]

In [3]:
df_train = pd.read_csv('../kaggle/train_feats.csv')
df_train = df_train.fillna(0)

df_train.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,session_label,volume,mid_price,liquidity_imbalance,matched_imbalance,size_imbalance,imbalance_intensity,matched_intensity,reference_price_far_price_imb,reference_price_near_price_imb,reference_price_ask_price_imb,reference_price_bid_price_imb,reference_price_wap_imb,far_price_near_price_imb,far_price_ask_price_imb,far_price_bid_price_imb,far_price_wap_imb,near_price_ask_price_imb,near_price_bid_price_imb,near_price_wap_imb,ask_price_bid_price_imb,ask_price_wap_imb,bid_price_wap_imb,price_spread,market_urgency,depth_pressure,price_pressure,imbalance_with_flag,imbalance_momentum,spread_intensity,matched_size_ret_1,matched_size_ret_2,matched_size_ret_3,matched_size_ret_4,matched_size_ret_5,matched_size_ret_6,imbalance_size_ret_1,imbalance_size_ret_2,imbalance_size_ret_3,imbalance_size_ret_4,imbalance_size_ret_5,imbalance_size_ret_6,bid_size_ret_1,bid_size_ret_2,bid_size_ret_3,bid_size_ret_4,bid_size_ret_5,bid_size_ret_6,ask_size_ret_1,ask_size_ret_2,ask_size_ret_3,ask_size_ret_4,ask_size_ret_5,ask_size_ret_6,reference_price_ret_1,reference_price_ret_2,reference_price_ret_3,reference_price_ret_4,reference_price_ret_5,reference_price_ret_6,ask_price_ret_1,ask_price_ret_2,ask_price_ret_3,ask_price_ret_4,ask_price_ret_5,ask_price_ret_6,bid_price_ret_1,bid_price_ret_2,bid_price_ret_3,bid_price_ret_4,bid_price_ret_5,bid_price_ret_6,market_urgency_ret_1,market_urgency_ret_2,market_urgency_ret_3,market_urgency_ret_4,market_urgency_ret_5,market_urgency_ret_6,imbalance_momentum_ret_1,imbalance_momentum_ret_2,imbalance_momentum_ret_3,imbalance_momentum_ret_4,imbalance_momentum_ret_5,imbalance_momentum_ret_6,size_imbalance_ret_1,size_imbalance_ret_2,size_imbalance_ret_3,size_imbalance_ret_4,size_imbalance_ret_5,size_imbalance_ret_6,imbalance_flag_diff,all_prices_mean,all_sizes_mean,all_prices_std,all_sizes_std,all_prices_skew,all_sizes_skew,all_prices_kurt,all_sizes_kurt,ask_price_bid_price_wap_imb2,ask_price_bid_price_reference_price_imb2,ask_price_wap_reference_price_imb2,bid_price_wap_reference_price_imb2,matched_size_bid_size_ask_size_imb2,matched_size_bid_size_imbalance_size_imb2,matched_size_ask_size_imbalance_size_imb2,bid_size_ask_size_imbalance_size_imb2,dow,seconds,minute,global_median_size,global_std_size,global_ptp_size,global_median_price,global_std_price,global_ptp_price
0,0,0,0,3180602.80,1,0.999812,13380277.0,0.0,0.0,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,1,69144.53,0.999919,0.754340,-0.615890,7.141326,45.999340,193.51172,0.0,0.0,-0.000107,0.000000,-0.000094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000107,0.000013,-9.400884e-05,0.000214,0.000161,0.0,680.649000,3180602.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999912,4157506.00,0.000117,6324881.0,0.042932,1.695159,0.000000,2.775961,0.138298,-1.927541e+12,0.138298,-1.693354e+12,255.368410,3.269177,3.215423,59.816770,0,0,0,39479.490,113601.550,5898989.50,1.999734,0.003419,0.017414
1,1,0,0,166603.90,-1,0.999896,1642214.2,0.0,0.0,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,1,23838.13,1.000278,-0.728750,-0.815787,0.156905,6.988967,68.89023,0.0,0.0,-0.000382,0.000000,-0.000052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000382,0.000330,-5.200270e-05,0.000764,-0.000557,0.0,127.285385,-166603.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000113,458164.06,0.000368,792759.4,1.897537,1.949990,3.617292,3.819817,6.346154,-6.881500e+12,6.346154,9.367487e+11,93.345870,9.032273,10.107002,8.404237,0,0,0,25830.525,6

In [4]:
df_valid = pd.read_csv('../kaggle/valid_feats.csv')
df_valid = df_valid.fillna(0)

df_valid.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,session_label,volume,mid_price,liquidity_imbalance,matched_imbalance,size_imbalance,imbalance_intensity,matched_intensity,reference_price_far_price_imb,reference_price_near_price_imb,reference_price_ask_price_imb,reference_price_bid_price_imb,reference_price_wap_imb,far_price_near_price_imb,far_price_ask_price_imb,far_price_bid_price_imb,far_price_wap_imb,near_price_ask_price_imb,near_price_bid_price_imb,near_price_wap_imb,ask_price_bid_price_imb,ask_price_wap_imb,bid_price_wap_imb,price_spread,market_urgency,depth_pressure,price_pressure,imbalance_with_flag,imbalance_momentum,spread_intensity,matched_size_ret_1,matched_size_ret_2,matched_size_ret_3,matched_size_ret_4,matched_size_ret_5,matched_size_ret_6,imbalance_size_ret_1,imbalance_size_ret_2,imbalance_size_ret_3,imbalance_size_ret_4,imbalance_size_ret_5,imbalance_size_ret_6,bid_size_ret_1,bid_size_ret_2,bid_size_ret_3,bid_size_ret_4,bid_size_ret_5,bid_size_ret_6,ask_size_ret_1,ask_size_ret_2,ask_size_ret_3,ask_size_ret_4,ask_size_ret_5,ask_size_ret_6,reference_price_ret_1,reference_price_ret_2,reference_price_ret_3,reference_price_ret_4,reference_price_ret_5,reference_price_ret_6,ask_price_ret_1,ask_price_ret_2,ask_price_ret_3,ask_price_ret_4,ask_price_ret_5,ask_price_ret_6,bid_price_ret_1,bid_price_ret_2,bid_price_ret_3,bid_price_ret_4,bid_price_ret_5,bid_price_ret_6,market_urgency_ret_1,market_urgency_ret_2,market_urgency_ret_3,market_urgency_ret_4,market_urgency_ret_5,market_urgency_ret_6,imbalance_momentum_ret_1,imbalance_momentum_ret_2,imbalance_momentum_ret_3,imbalance_momentum_ret_4,imbalance_momentum_ret_5,imbalance_momentum_ret_6,size_imbalance_ret_1,size_imbalance_ret_2,size_imbalance_ret_3,size_imbalance_ret_4,size_imbalance_ret_5,size_imbalance_ret_6,imbalance_flag_diff,all_prices_mean,all_sizes_mean,all_prices_std,all_sizes_std,all_prices_skew,all_sizes_skew,all_prices_kurt,all_sizes_kurt,ask_price_bid_price_wap_imb2,ask_price_bid_price_reference_price_imb2,ask_price_wap_reference_price_imb2,bid_price_wap_reference_price_imb2,matched_size_bid_size_ask_size_imb2,matched_size_bid_size_imbalance_size_imb2,matched_size_ask_size_imbalance_size_imb2,bid_size_ask_size_imbalance_size_imb2,dow,seconds,minute,global_median_size,global_std_size,global_ptp_size,global_median_price,global_std_price,global_ptp_price
0,0,436,0,0.0,0,1.000268,12874820.0,0.0,0.0,0.999911,11182.00,1.000089,11184.00,1.0,6.630421,1,22366.00,1.000000,-0.000089,-1.000000,0.999821,0.00000,575.64250,0.0,0.0,0.000089,0.000178,0.000134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000089,0.000044,-0.000045,0.000178,-1.591702e-08,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000067,3224296.5,0.000152,6433684.5,0.759262,1.999996,0.000000,3.999987,1.000000,1.005618e+00,2.011236,3.011236e+00,6.431818e+06,1150.388000,1150.182000,0.000179,1,0,0,39479.490,113601.550,5898989.50,1.999734,0.003419,0.017414
1,1,436,0,1378667.5,1,0.999853,2806215.2,0.0,0.0,0.999853,801.20,1.000552,3006.60,1.0,-4.770160,1,3807.80,1.000203,-0.579180,-0.341120,0.266480,362.06403,736.96497,0.0,0.0,-0.000349,0.000000,-0.000074,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000349,0.000276,-0.000074,0.000699,-4.048465e-04,0.0,963.68854,1378667.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000064,1047172.6,0.000332,1340310.4,1.756207,0.884692,3.037954,-1.132242,3.755102,6.296032e+12,3.755102,-1.324058e+12,1.271066e+03,1.036057,1.037718,623.769300,1,0,0,25830.525,64297.402,

In [12]:
feat_cols = [col for col in df_train.columns if col not in ['target', 'date_id', 'dow']]

standarize_cols = [col for col in feat_cols if col not in ['stock_id', 'seconds_in_bucket', 'imbalance_buy_sell_flag', 'session_label', 'seconds', 'minute']]

df_train_feat = df_train[feat_cols]
scaler = StandardScaler()
df_train_feat[standarize_cols] = scaler.fit_transform(df_train_feat[standarize_cols])
train_dataset = MyDataset(torch.from_numpy(df_train_feat.values), torch.from_numpy(df_train['target'].values))

df_valid_feat = df_valid[feat_cols]
df_valid_feat[standarize_cols] = scaler.transform(df_valid_feat[standarize_cols])
valid_dataset = MyDataset(torch.from_numpy(df_valid_feat.values), torch.from_numpy(df_valid['target'].values))

print(train_dataset.X.shape, train_dataset.y.shape)
print(valid_dataset.X.shape, valid_dataset.y.shape)

torch.Size([4742893, 128]) torch.Size([4742893])
torch.Size([494999, 128]) torch.Size([494999])


In [18]:
# define a neural network model of 2 layers
# first layer is a non-linear layer with m neurons
# second layer is a linear layer with n neuron

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        x = self.flatten(x)
        output = self.linear_relu_stack(x)
        return output

model = NeuralNetwork()

In [19]:
from sklearn.metrics import mean_absolute_error

# define a function to train the model
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.float())
        loss = criterion(output, target.float())
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

# define a function for validation
def validation(model, val_loader, criterion):
    model.eval()
    validation_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data.float())
            validation_loss += criterion(output, target.float()).item()
            y_true.extend(target.tolist())
            y_pred.extend(output.tolist())
            
    validation_loss /= len(val_loader.dataset)
    print(f"\nValidation set: Average loss: {validation_loss:.6f}")
    print(f"\nValidation set: Mean Average Error: {mean_absolute_error(y_true, y_pred):.6f}\n")

# define a function to predict the target
def predict(model, pred_loader):
    model.eval()
    y_pred = []
    with torch.no_grad():
        for data, target in pred_loader:
            output = model(data.float())
            y_pred.extend(output.tolist())
            
    return y_pred

In [20]:
# print number of data points and number of features
print(f"Number of data points: {train_dataset.X.shape[0]}")
print(f"Number of features: {train_dataset.X.shape[1]}")

# train the model with train_dataset
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=2048, shuffle=True)
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(1, 3):
    train(model, train_loader, optimizer, criterion, epoch)
    validation(model, val_loader, criterion)


Number of data points: 4742893
Number of features: 128
Train Epoch: 1 [0/4742893 (0%)]	Loss: 183.453156
Train Epoch: 1 [204800/4742893 (4%)]	Loss: 102.883232
Train Epoch: 1 [409600/4742893 (9%)]	Loss: 97.860550
Train Epoch: 1 [614400/4742893 (13%)]	Loss: 97.209587
Train Epoch: 1 [819200/4742893 (17%)]	Loss: 88.297920
Train Epoch: 1 [1024000/4742893 (22%)]	Loss: 85.547958
Train Epoch: 1 [1228800/4742893 (26%)]	Loss: 109.386353
Train Epoch: 1 [1433600/4742893 (30%)]	Loss: 87.934937
Train Epoch: 1 [1638400/4742893 (35%)]	Loss: 86.558990
Train Epoch: 1 [1843200/4742893 (39%)]	Loss: 89.566422
Train Epoch: 1 [2048000/4742893 (43%)]	Loss: 77.850868
Train Epoch: 1 [2252800/4742893 (47%)]	Loss: 87.915054
Train Epoch: 1 [2457600/4742893 (52%)]	Loss: 84.177773
Train Epoch: 1 [2662400/4742893 (56%)]	Loss: 76.049026
Train Epoch: 1 [2867200/4742893 (60%)]	Loss: 86.807907
Train Epoch: 1 [3072000/4742893 (65%)]	Loss: 86.456741
Train Epoch: 1 [3276800/4742893 (69%)]	Loss: 89.771408
Train Epoch: 1 [3481

In [21]:
# save the model
torch.save(model.state_dict(), "model_fcn_2layer.pth")